<a href="https://colab.research.google.com/github/jjbmsda/langmodel/blob/main/Flask_GenerateTextWeb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from flask import Flask, render_template, request
import torch
from kogpt2.pytorch_kogpt2 import get_pytorch_kogpt2_model
from transformers import PreTrainedTokenizerFast, TextGenerationPipeline, GenerationConfig

app = Flask(__name__)

tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                                                    bos_token='</s>', eos_token='</s>', unk_token='<unk>',
                                                    pad_token='<pad>', mask_token='<mask>')
model = get_pytorch_kogpt2_model()

generation_config = GenerationConfig(
    max_length=1000,
    do_sample=True,
    top_p=0.95,
    top_k=50,
    temperature=1.0,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id
)

text_generator = TextGenerationPipeline(model=model, tokenizer=tokenizer, **generation_config.to_dict())

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/generate_text', methods=['POST'])
def generate_text():
    input_text = request.form['input_text']
    generated_text = text_generator(input_text)[0]["generated_text"]
    output_text = generated_text.replace(input_text, "").strip()
    return output_text

if __name__ == '__main__':
    app.run(debug=True)


ModuleNotFoundError: ignored